In [ ]:
!pip install python-google-places

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python-google-places: filename=python_google_places-1.4.2-py3-none-any.whl size=13600 sha256=1e0dff0da0d01b635a2405a07733a8e394bcc076da2793232e2ed54b3d14d7ad
  Stored in directory: /root/.cache/pip/wheels/9f/11/19/361da17e9681d96dd5b4b2f166df413cb9dc11373d2f8565cf
Successfully built python-google-places


In [ ]:
from googleplaces import GooglePlaces, types, lang
import pandas as pd
import numpy as np

In [ ]:
mypath ='/content/drive/MyDrive/295A Project/data'
API_KEY = ''

In [ ]:
google_places = GooglePlaces(API_KEY)
def googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type):
    """Transform query results from Google API to pandas dataframe"""
    places_data_list = []
    query_result = google_places.nearby_search(
    lat_lng={'lat': lat, 'lng': lng}, 
    #location=(lat, lng),
    keyword=keyword,
    radius=radius)    
    for place in query_result.places: 
        place.get_details()
        places_data_list.append([place.place_id, place.name, float(place.geo_location['lat']),float(place.geo_location['lng']), place.rating, place.url])
    #default query only returns the first 20
    while query_result.has_next_page_token :
        query_result= google_places.nearby_search(pagetoken=query_result.next_page_token)
        for place in query_result.places:
            place.get_details()
            places_data_list.append([place.place_id, place.name, float(place.geo_location['lat']),float(place.geo_location['lng']), place.rating, place.url])
    df = pd.DataFrame(places_data_list, columns = ['ID','Name', 'latitude', 'longitude', 'Rating', 'Url'])
    return df

In [ ]:
location = 'Palo Alto'
lat, lng = 37.44220793084094, -122.14317583273053
radius = 1500
keyword = 'charging station'
df = googleplaces_query_to_dataframe (lng, lat, keyword, radius)
df.shape

(17, 6)

In [ ]:
df.head()

,ID,Name,latitude,longitude,Rating,Url
0,ChIJUQFR4wG7j4AR7LdpWWsgVaM,Powerflex Charging Station,37.438797,-122.134121,0,https://maps.google.com/?cid=11769348846636349420
1,ChIJiVr2Cgi7j4ARWohyzpN09a0,Tesla Supercharger,37.446566,-122.162295,4.2,https://maps.google.com/?cid=12535053316028270682
2,ChIJq-YIzju7j4ARSfzqjgfWW34,Powerflex Charging Station,37.444521,-122.160017,0,https://maps.google.com/?cid=9105106399610469449
3,ChIJGe7BQD-7j4ARG6JeLXxYrz0,ChargePoint Charging Station,37.449806,-122.156487,0,https://maps.google.com/?cid=4444868647598334491
4,ChIJ4Ydlyzi7j4ARXUGqIa4tpf8,Powerflex Charging Station,37.449094,-122.157694,0,https://maps.google.com/?cid=18421180076741247325


In [ ]:
upperleft = [37.425045862307954, -122.19381908622464]
upperright = [37.460518352884044, -122.14193347005475]
lowerleft = [37.38962953738927, -122.12820056019407]
lowerright = [37.434351837072924, -122.08494189413288]
lat_list = np.arange(37.425045862307954, 37.434351837072924, 0.025)
lng_list = np.arange(-122.19381908622464, -122.08494189413288,0.025)
xx,yy = np.meshgrid(lat_list,lng_list)

In [ ]:
xx.shape

(5, 1)

In [ ]:
yy.shape

(5, 1)

In [ ]:
coords = np.array([xx, yy]).reshape(2,-1).T
coords.shape

(5, 2)

In [ ]:
coords = pd.DataFrame(coords, columns = ['Lat', 'Lon'])
coords.head()

,Lat,Lon
0,37.425046,-122.193819
1,37.425046,-122.168819
2,37.425046,-122.143819
3,37.425046,-122.118819
4,37.425046,-122.093819


In [ ]:
def poi_scrape(centroids_df, keyword, output_file_name, *place_type):
    """ Scraping points of interests given a dataframe that contains geo-coordinates of centroids """
    df = pd.DataFrame()
    row_index = 0
    radius = 1500
    for index, row in centroids_df.iterrows():
        lat, lng= row['Lat'], row['Lon']   
        current_length = len(df)
        df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))
        print('scraped row ', row_index, ' and found', len(df)-current_length, 'results')
        row_index += 1
    print('# results before dup drop', len(df))
    df_drop_dup = df.drop_duplicates('ID')
    print('# results after dup drop', len(df_drop_dup))
    df_drop_dup.to_excel(mypath+'/raw/'+output_file_name)
    return df_drop_dup

In [ ]:
df_charging = poi_scrape(coords, 'charging station','TRT_charging.xlsx')

<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))
<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 11 results
scraped row  1  and found 12 results
scraped row  2  and found 18 results
scraped row  3  and found 7 results
scraped row  4  and found 3 results
# results before dup drop 51
# results after dup drop 38


<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))
<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))
<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
df_foodcourt = poi_scrape(coords, 'food court', 'TRT_foodcourt.xlsx')

<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 4 results


<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  1  and found 6 results


<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  2  and found 2 results


<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  3  and found 2 results
scraped row  4  and found 3 results
# results before dup drop 17
# results after dup drop 12


<ipython-input-25-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
df_parking = poi_scrape(coords, 'parking', 'TRT_parking_type_parking.xlsx', 'parking')

<ipython-input-31-38856c3ee691>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 1 results


<ipython-input-31-38856c3ee691>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  1  and found 14 results


<ipython-input-31-38856c3ee691>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  2  and found 13 results


<ipython-input-31-38856c3ee691>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  3  and found 0 results
scraped row  4  and found 1 results
# results before dup drop 29
# results after dup drop 29


<ipython-input-31-38856c3ee691>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
# drop the "parks" observations
df_parking2 = df_parking[df_parking.Name.str.endswith('Park') == False]
df_parking2.shape

(29, 6)

In [ ]:
df_parking2.to_excel(mypath+'/raw/'+'TRT_parking_lots_2.xlsx')

In [ ]:
df_shopping = poi_scrape(coords, 'shopping center', 'TRT_shopping.xlsx')

<ipython-input-37-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 1 results


<ipython-input-37-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  1  and found 2 results


<ipython-input-37-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  2  and found 3 results


<ipython-input-37-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  3  and found 3 results
scraped row  4  and found 3 results
# results before dup drop 12
# results after dup drop 9


<ipython-input-37-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
#df_restaurant = poi_scrape(coords, 'restaurant', 'TRT_restaurant.xlsx')

In [ ]:
df_grocery = poi_scrape(coords, 'grocery store', 'TRT_grocery.xlsx')

<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 2 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  1  and found 3 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  2  and found 8 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  3  and found 5 results
scraped row  4  and found 4 results
# results before dup drop 22
# results after dup drop 15


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
df_gas = poi_scrape(coords, 'gas station', 'TRT_gas.xlsx')

<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  0  and found 2 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  1  and found 1 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  2  and found 5 results


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


scraped row  3  and found 9 results
scraped row  4  and found 7 results
# results before dup drop 24
# results after dup drop 15


<ipython-input-47-1e30a7e50e2e>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(googleplaces_query_to_dataframe (lng, lat, keyword, radius, *place_type ))


In [ ]:
#df_university = poi_scrape(coords, 'university', 'TRT_university.xlsx')